Автор материала: Юрий Кашницкий (@yorko в Slack ODS). Материал распространяется на условиях лицензии Creative Commons CC BY-NC-SA 4.0.

# Домашнее задание 6 (демо). Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина

In [15]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics.regression import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

Будем работать с набором данных по качеству белого вина (репозиторий UCI). Загружаем данные.

In [16]:
data = pd.read_csv('wine_quality.csv')

In [17]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [27]:
data.columns
y = data['quality']
X = data.drop('quality', axis=1)

Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [31]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Линейная регрессия
Обучите простую линейную регрессию.

In [32]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Вопрос 1: Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?

In [33]:
y_train_pred = linreg.predict(X_train_scaled)
y_test_pred = linreg.predict(X_test_scaled)

In [34]:
print(f'Mean squared error (train): {mean_squared_error(y_train, y_train_pred)}')
print(f'Mean squared error (test): {mean_squared_error(y_test, y_test_pred)}')

Mean squared error (train): 0.5648982958116774
Mean squared error (test): 0.562561553622836


Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.

Вопрос 2: Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?

In [86]:
coefficients = np.abs(linreg.coef_.ravel())
cols = X_train.columns

linreg_coef = pd.DataFrame(data=coefficients.reshape(11, 1), columns=['coef'], index=cols)

In [89]:
linreg_coef.sort_values(by='coef', ascending=False)

,coef
density,0.400258
residual sugar,0.390505
alcohol,0.255748
volatile acidity,0.191070
pH,0.087720
free sulfur dioxide,0.073669
sulphates,0.056607
fixed acidity,0.055695
total sulfur dioxide,0.021225
chlorides,0.020196


## Lasso

In [90]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=17,
      selection='cyclic', tol=0.0001, warm_start=False)

Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?

In [93]:
coefficients = np.abs(lasso1.coef_.ravel())
cols = X_train.columns

lasso1_coef = pd.DataFrame(data=coefficients.reshape(11, 1), columns=['coef'], index=cols)
lasso1_coef.sort_values(by='coef', ascending=False)

,coef
alcohol,0.366044
residual sugar,0.197612
volatile acidity,0.190913
density,0.134286
free sulfur dioxide,0.062599
sulphates,0.031170
pH,0.030322
chlorides,0.025076
total sulfur dioxide,0.007980
fixed acidity,0.000000


Теперь определите лучшее значение α в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.

In [95]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, cv=5)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
        n_alphas=100, n_jobs=None, normalize=False, positive=False,
        precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
        verbose=False)

In [96]:
lasso_cv.alpha_

0.006593188271333548

Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина.
Вопрос 3: Какой признак "обнулился первым" в настроенной модели LASSO?

In [102]:
lasso_cv_coef = pd.DataFrame(data=np.abs(lasso_cv.coef_.reshape(11, 1)), columns=['coef'], index=cols)
lasso_cv_coef.sort_values(by='coef', ascending=False)

,coef
alcohol,0.347484
residual sugar,0.235317
volatile acidity,0.192937
density,0.182113
free sulfur dioxide,0.066213
sulphates,0.037970
pH,0.037813
chlorides,0.025194
total sulfur dioxide,0.012434
fixed acidity,0.000000


Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.
Вопрос 4: Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?

In [109]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, lasso_cv.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_test, lasso_cv.predict(X_test_scaled)))

Mean squared error (train): 0.567
Mean squared error (test): 0.567


# Случайный лес
Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.

In [117]:
forest = RandomForestRegressor(random_state=17, n_estimators=100) # Ваш код здесь
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=17, verbose=0,
                      warm_start=False)

In [126]:
print("Mean squared error (train): %.3f" % cross_val_score(
    forest,
    X_train_scaled, 
    y_train, cv=5).mean()
     )

print("Mean squared error (test): %.3f" % cross_val_score(
    forest,
    X_test_scaled, 
    y_test, cv=5).mean()
     )

Mean squared error (train): 0.478
Mean squared error (test): 0.413


In [124]:
cross_val_score(forest, X_train_scaled, y_train, cv=5)


array([0.5150696 , 0.47828465, 0.47076011, 0.46241234, 0.46252979])

Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.

In [129]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

cv_forest = GridSearchCV(RandomForestRegressor(n_estimators=100), param_grid=forest_params, cv=5)
cv_forest.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [10, 11, 12, 13, 14, 

In [130]:
cv_forest.best_params_, cv_forest.best_scores_

AttributeError: 'GridSearchCV' object has no attribute 'best_scores_'

Вопрос 6: Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?

In [ ]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, cv_forest.predict(X_train_scaled)))
print("Mean squared error (cv): %.3f" % cross_val_score(cv_forest,
    X_train_scaled, 
    y_train, cv=5).mean()))
print("Mean squared error (test): %.3f" % mean_squared_error(y_test, cv_forest.predict(X_test_scaled)))